# Table of Contents

* [Introduction](#introduction)
* [Imports](#imports)
    - [Standard](#standard)
    - [Import OS and Get/Make Directories](#import-os)
    - [Tensorflow & Keras](#tensorflow)
    - [Other Tools](#other-tools)
* [Data](#data)
    - [Convert Images to Tensors](#convert-images)
* [Translate Images](#translate-images)
* [Save Translation](#save-translation)
* [Test Set](#test-set)
    - [Data](#test-data)
    - [Translate Images](#test-translate-images)
    - [Save Translation](#test-save-translation)

# Introduction <a id="introduction"></a>

After the cycleGAN has been trained in train_watermark_removal.ipynb, then next step is to apply the cycleGAN to the train and test sets of food images and remove watermarks. As mentioned before, only generator G_n that translates watermarked images to unwatermarked images, is necessary for this. 

# Imports <a id="imports"></a>

## Standard <a id="standard"></a>

In [ ]:
import numpy as np
import pandas as pd

## Import OS and Get/Make Directories <a id="import-os"></a>

In [ ]:
import os 
from pathlib import Path
cwd = os.path.abspath(os.getcwd())
cwd

In [ ]:
data_dir = Path("/".join(cwd.split("/"))) / "data"

data_dir

In [ ]:
train_dir = data_dir / "train_set"

train_dir

In [ ]:
test_dir = data_dir / "test_set"

test_dir

In [ ]:
model_dir = Path(cwd) / "models"

model_dir

## Tensorflow & Keras <a id="tensorflow"></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras import layers
from keras.models import Sequential, Model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Other Tools <a id="other-tools"></a>

In [ ]:
from tqdm import tqdm
from PIL import Image

# Data <a id="data"></a>

## Convert Images to Tensors <a id="convert-images"></a>

In [ ]:
original_images = os.listdir(train_dir)
if ".DS_Store" in original_images:
  original_images.remove(".DS_Store")

tensor_images = []
for img_name in tqdm(original_images):
  img_rgb = cv2.cvtColor(cv2.imread(filename=str(train_dir / img_name)), cv2.COLOR_BGR2RGB)
  tensor_images.append(tf.convert_to_tensor(img_rgb, dtype=tf.float32))


tensor_images = np.asarray(tensor_images)
tensor_images.shape

## Normalise Tensors <a id="normalise-tensors"></a>

In [ ]:
def normalize(image):
  image = tf.cast(image, tf.float32)
  image = (image / 127.5) - 1
  return image

In [ ]:
normalised_tensors = []
for tensor_img in tqdm(tensor_images):
  normalised_tensors.append(normalize(tensor_img))

normalised_tensors = np.asarray(normalised_tensors)
normalised_tensors.shape

# Translate Images <a id="translate-images"></a>

In [ ]:
remove_watermarks = tf.keras.models.load_model(model_dir / "generator_n")

remove_watermarks.summary()

In [ ]:
def crop_image_height(image):
  if image.shape[1] > 256:
    max_height = image.shape[1]
    max_slice = max_height - 257
    start_slice = np.random.randint(0, max_slice) if max_slice > 0 else 0
    end_slice = start_slice + 256
    return image[:,start_slice:end_slice,:]
  else: 
    return image

In [ ]:
def crop_image_width(image):
  if image.shape[0] > 256:
    max_width = image.shape[0]
    max_slice = max_width - 257
    start_slice = np.random.randint(0, max_slice) if max_slice > 0 else 0
    end_slice = start_slice + 256
    return image[start_slice:end_slice,:,:]
  else: 
    return image

In [ ]:
mark_removed_tensors = []

for t_img in tqdm(normalised_tensors):
    cropped_img = crop_image_height(t_img)
    cropped_img = crop_image_width(cropped_img)
    mark_removed_tensors.append(remove_watermarks.predict(cropped_img[np.newaxis,:]))

# Save Translation <a id="save-translation"></a>

The code block below is implemented from this [StackOverflow answer](https://stackoverflow.com/questions/57338091/how-to-save-tensor-as-a-image).

In [ ]:
filenames = os.listdir(train_dir)
if ".DS_Store" in filenames:
  filenames.remove(".DS_Store")

no_of_files = len(filenames)
no_of_files

In [ ]:
train_mrk_removed_dir = data_dir / "train_set_watermark_removed"
if train_mrk_removed_dir.exists() == False:
  os.mkdir(train_mrk_removed_dir)

data_dir

The code block below is implemented from this [StackOverflow answer](https://stackoverflow.com/questions/57338091/how-to-save-tensor-as-a-image).

In [ ]:
with tf.Session() as sess:
    init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
    sess.run(init)
    tf.train.start_queue_runners()
    for i in tqdm(range(no_of_files)):
        img_tnsr = mark_removed_tensors[i]
        img = sess.run(img_tnsr)
        img = Image.fromarray(img)
        img.save(test_mrk_removed_dir / filenames[i])

# Test Set <a id="test-set"></a>

## Data <a id="test-data"></a>

In [ ]:
original_images = os.listdir(train_dir)
if ".DS_Store" in original_images:
  original_images.remove(".DS_Store")

tensor_images = []
for img_name in tqdm(original_images):
  img_rgb = cv2.cvtColor(cv2.imread(filename=str(train_dir / img_name)), cv2.COLOR_BGR2RGB)
  tensor_images.append(tf.convert_to_tensor(img_rgb, dtype=tf.float32))


tensor_images = np.asarray(tensor_images)
tensor_images.shape

In [ ]:
normalised_tensors = []
for tensor_img in tqdm(tensor_images):
  normalised_tensors.append(normalize(tensor_img))

normalised_tensors = np.asarray(normalised_tensors)
normalised_tensors.shape

## Translate Images <a id="test-translate-images"></a>

In [1]:
remove_watermarks = tf.keras.models.load_model(model_dir / "generator_f")

remove_watermarks.summary()

NameError: name 'tf' is not defined

In [ ]:
mark_removed_tensors = []

for t_img in tqdm(normalised_tensors):
  cropped_img = crop_image_height(t_img)
  cropped_img = crop_image_width(cropped_img)
  mark_removed_tensors.append(remove_watermarks.predict(cropped_img[np.newaxis,:]))

## Save Translation <a id="test-save-translation"></a>

In [ ]:
filenames = os.listdir(test_dir)
if ".DS_Store" in filenames:
  filenames.remove(".DS_Store")
no_of_files = len(filenames)

In [ ]:
test_mrk_removed_dir = data_dir / "test_set_watermark_removed"
if test_mrk_removed_dir.exists() == False:
  os.mkdir(test_mrk_removed_dir)

test_mrk_removed_dir

In [ ]:
with tf.Session() as sess:
    init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
    sess.run(init)
    tf.train.start_queue_runners()
    for i in tqdm(range(no_of_files)):
        img_tnsr = mark_removed_tensors[i]
        img = sess.run(img_tnsr)
        img = Image.fromarray(img)
        img.save(test_mrk_removed_dir / filenames[i])